In [84]:
#Importing necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

In [86]:
# Load IHEC metadata file
ihec_data = pd.read_csv("IHEC_sample_metadata_harmonization.v1.3_extended.csv")
ihec_data

,EpiRR,project,harmonized_biomaterial_type,harmonized_sample_label,harmonized_sample_ontology_intermediate,harmonized_sample_ontology_intermediate_color,harmonized_sample_disease_high,harmonized_sample_disease_intermediate,harmonized_EpiRR_status,epiATLAS_status,...,automated_experiments_ChIP-Seq_H3K36me3,automated_experiments_ChIP-Seq_H3K4me1,automated_experiments_ChIP-Seq_H3K4me3,automated_experiments_ChIP-Seq_H3K9me3,automated_experiments_WGBS_standard,automated_experiments_WGBS_PBAT,automated_experiments_RNA-Seq_mRNA-Seq,automated_experiments_RNA-Seq_total-RNA-Seq,epirr_id_without_version,EpiRR_ordering
0,IHECRE00001032.6,NIH Roadmap Epigenomics,primary tissue,Brain,brain,"185,126,30",Healthy/None,None,Partial,Partial,...,aa8df9df-e3b8-483c-9472-28f4928aa4b6,df1f1ce4-1a3a-4759-9431-f00274a7749b,e17c375b-300c-4045-9799-fc9bb5e8d78a,035e98a3-b88a-477c-9263-0b08641a3364,imputed,NaN,NaN,NaN,IHECRE00001032,1
1,IHECRE00001053.7,NIH Roadmap Epigenomics,primary tissue,Brain,brain,"185,126,30",Healthy/None,None,Partial,Partial,...,imputed,imputed,51ea052b-270e-4ae2-ab39-50b171bd58a5,imputed,imputed,NaN,NaN,NaN,IHECRE00001053,2
2,IHECRE00002063.1,CEEHRC,primary tissue,Brain Amygdala,brain,"185,126,30",Healthy/None,None,Complete,Complete,...,5cbd3cd5-dc33-49dc-ae71-41bb3114f33a,03cd26f6-8245-4dba-a43d-b085dc5fbdba,7418f6bc-6e40-4cd9-9c59-6013c4a60b00,29eb7dda-7f49-49e4-a68a-685ebc852d97,ad373be4-9739-4cbd-9f33-0ddd3ce45304,NaN,NaN,16319258-30fb-4402-a2c7-ae2b4f6f74be,IHECRE00002063,3
3,IHECRE00002086.1,CEEHRC,primary tissue,Brain Amygdala,brain,"185,126,30",Healthy/None,None,Complete,Complete,...,102990be-2f15-4965-9094-d6b665cc5fb0,469e2b5a-9cc0-4ba2-a1af-32ae688b6e56,e56e7efe-813c-4380-b8af-078cacdb962a,982b1a46-0bf8-457b-a213-1cec9bf76346,7ad1ee11-bdd1-4607-a385-226e88c6f940,NaN,NaN,da4afbf6-bcc1-4d3c-8eac-5e36d1a4b805,IHECRE00002086,4
4,IHECRE00001973.1,CEEHRC,primary tissue,Brain Amygdala,brain,"185,126,30",Healthy/None,None,Complete,Complete,...,f006c884-d6f7-470d-8586-314761e38a9f,fa7392b1-17c1-4691-ba8f-d211a0f93db5,12487572-adf6-40c8-9745-66a141c14768,6e93a1e0-8828-4ce5-83f8-36249902bda2,0fad140f-c9a5-4da6-b789-485e11b7bdf3,NaN,NaN,cd02f672-1b6d-4841-83d9-c7d187a2a7f5,IHECRE00001973,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2274,IHECRE00000975.8,NIH Roadmap Epigenomics,primary cell,Melanocyte Foreskin,melanocyte,"113,41,32",Healthy/None,None,Partial,Partial,...,401fde27-8902-40c0-8a71-c1861d67a913,28dce804-616e-461a-a241-f86f38849801,97a1e865-f6f5-4bb4-a302-8bb6deedc0ef,0cf75112-954a-4d05-98c6-8191f76b9e0b,imputed,NaN,NaN,NaN,IHECRE00000975,2275
2275,IHECRE00001026.7,NIH Roadmap Epigenomics,primary cell,Melanocyte Foreskin,melanocyte,"113,41,32",Healthy/None,None,Partial,Complete_imputed,...,81227768-8af0-4c8e-8b0b-e1308eaefa55,12e4d18a-69a4-49cf-aa63-7cd9c6b4f158,160994fa-0800-4eaf-ac95-c9c39242a36b,997e79ff-8ff2-489c-8380-15cef3b2293f,imputed,NaN,fafd4afa-b115-4895-922c-141e82bffa60,NaN,IHECRE00001026,2276
2276,IHECRE00000928.7,NIH Roadmap Epigenomics,primary tissue,Ovarian Tissue,ovary,"181,104,96",Healthy/None,None,Complete,Partial,...,NaN,NaN,NaN,NaN,NaN,49594f8a-8dd8-47f3-989d-3aa48f1dd6c4,d1c68642-5108-4497-b725-e555cc161c3b,NaN,IHECRE00000928,2277
2277,IHECRE00004688.5,ENCODE,primary tissue,Uterine Tissue,uterus,"135,105,48",Healthy/None,None,Partial,Complete_imputed,...,imputed,0785d70e-043f-4e7a-bc91-6d59853273fa,a1984693-dd60-4afe-9120-d47d69e32402,ce89c637-c90f-4659-b060-617a76f443f9,imputed,NaN,NaN,e0902bfa-5f1d-4320-a381-be7227f1b333,IHECRE00004688,2278


In [87]:
# Load binary feature matrix for each histone mark
'''Replace tsv file with "H3K27me3_CUTOFF10000_CUTOFF10000_metapeak_counts_binary.tsv", 
"H3K36me3_CUTOFF10000_CUTOFF10000_metapeak_counts_binary.tsv", "H3K4me1_CUTOFF10000_CUTOFF10000_metapeak_counts_binary.tsv",
"H3K4me3_CUTOFF10000_CUTOFF10000_metapeak_counts_binary.tsv", "H3K9me3_CUTOFF10000_CUTOFF10000_metapeak_counts_binary.tsv"
based on the histone mark you want to do the analysis on.
'''
data = pd.read_csv("H3K27ac_CUTOFF10000_CUTOFF10000_metapeak_counts_binary.tsv", sep = "\t")
data

,ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00000001.4.8e4a8d6c-26e0-4b8d-aa08-7875973212d4.pval0.01.500K.bfilt.narrowPeak,ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00000002.3.98a96310-7c2e-4c5c-911e-9bcd1d1a32bb.pval0.01.500K.bfilt.narrowPeak,ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00000004.3.08828406-540b-464b-b2f8-9402ccb1159b.pval0.01.500K.bfilt.narrowPeak,ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00000008.3.2e1ab747-5c43-4469-97d5-a2e674bc3ee6.pval0.01.500K.bfilt.narrowPeak,ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00000009.3.2dc8b941-fde9-464c-b465-4ae10f662f7a.pval0.01.500K.bfilt.narrowPeak,ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00000010.3.867d9ec3-9d9c-46a1-85cf-728b76d20f0a.pval0.01.500K.bfilt.narrowPeak,ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00000011.3.869f322e-d61a-4e6d-81c3-8e01a3a33de3.pval0.01.500K.bfilt.narrowPeak,ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00000013.3.21204e63-5580-42b6-aaaf-3888daa8192c.pval0.01.500K.bfilt.narrowPeak,ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00000014.2.6a99d923-7ab5-41be-874b-467d20e26e79.pval0.01.500K.bfilt.narrowPeak,ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00000016.3.90c9bd75-e378-4adf-8726-9bc0261986e3.pval0.01.500K.bfilt.narrowPeak,...,ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00004880.1.8e1e0bb1-2e06-4414-9be5-0544b4b88ab1.pval0.01.500K.bfilt.narrowPeak,ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00004883.1.892c9cb6-62f8-47be-8833-21d2a5f1a57f.pval0.01.500K.bfilt.narrowPeak,ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00004884.1.cdb41994-86e2-4949-bf91-2378fd47bfa2.pval0.01.500K.bfilt.narrowPeak,ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00004885.1.e0ffc3ad-8757-42f6-8d96-1d2f2847014b.pval0.01.500K.bfilt.narrowPeak,ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00004886.1.c7a8b983-d830-4761-b97a-1b16f0b15a0d.pval0.01.500K.bfilt.narrowPeak,ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00004888.1.fd25c27d-77c4-4595-b48c-e5f0d2d321a9.pval0.01.500K.bfilt.narrowPeak,ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00004894.1.8e262744-4f2b-4443-8aef-70ca1b496285.pval0.01.500K.bfilt.narrowPeak,ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00004897.1.63d2c246-6b70-4ed3-8656-25c6f860b51b.pval0.01.500K.bfilt.narrowPeak,ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00004900.1.b52b6048-f5e8-441d-8082-83c6ea7d797d.pval0.01.500K.bfilt.narrowPeak,ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00004903.1.f71ea030-5c25-4b10-8d23-afc537e49870.pval0.01.500K.bfilt.narrowPeak
chr1_840917_844971_n_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
chr1_999222_1001990_n_2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,1,0,0,0,0
chr1_1024892_1028188_n_3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
chr1_1135735_1138195_n_4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
chr1_1217473_1220087_n_5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chrY_21072393_21083950_n_19768,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
chrY_21135972_21142739_n_19769,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
chrY_21173280_21177987_n_19770,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
chrY_21238808_21243547_n_19771,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
# Create a dictionary mapping index to column name (sample identifiers)
dict = {}
for i in range(0, len(data.columns)):
    dict[i] = data.columns[i]

In [90]:
dict

{0: 'ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00000001.4.8e4a8d6c-26e0-4b8d-aa08-7875973212d4.pval0.01.500K.bfilt.narrowPeak',
 1: 'ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00000002.3.98a96310-7c2e-4c5c-911e-9bcd1d1a32bb.pval0.01.500K.bfilt.narrowPeak',
 2: 'ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00000004.3.08828406-540b-464b-b2f8-9402ccb1159b.pval0.01.500K.bfilt.narrowPeak',
 3: 'ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00000008.3.2e1ab747-5c43-4469-97d5-a2e674bc3ee6.pval0.01.500K.bfilt.narrowPeak',
 4: 'ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00000009.3.2dc8b941-fde9-464c-b465-4ae10f662f7a.pval0.01.500K.bfilt.narrowPeak',
 5: 'ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00000010.3.867d9ec3-9d9c-46a1-85cf-728b76d20f0a.pval0.01.500K.bfilt.narrowPeak',
 6: 'ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00000011.3.869f322e-d61a-4e6d-81c3-8e01a3a33de3.pval0.01.500K.bfilt.narrowPeak',
 7: 'ihec.chipseq.ihec-chipseq-containerv1.1.4.IHECRE00000013.

In [91]:
# Transpose the data so that rows are samples and columns are features
data = data.transpose()
data.reset_index(drop = True, inplace = True)
data

,chr1_840917_844971_n_1,chr1_999222_1001990_n_2,chr1_1024892_1028188_n_3,chr1_1135735_1138195_n_4,chr1_1217473_1220087_n_5,chr1_1297695_1303428_n_6,chr1_1356115_1362473_n_7,chr1_1398275_1400662_n_8,chr1_1418435_1435676_n_9,chr1_1539204_1541760_n_10,...,chrY_11063329_11066391_n_19763,chrY_12661354_12663911_n_19764,chrY_13477999_13480655_n_19765,chrY_14523175_14526479_n_19766,chrY_19075455_19078099_n_19767,chrY_21072393_21083950_n_19768,chrY_21135972_21142739_n_19769,chrY_21173280_21177987_n_19770,chrY_21238808_21243547_n_19771,chrY_21247559_21259006_n_19772
0,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,1,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1560,0,1,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1561,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1562,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
1563,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
# Split data into training and testing sets for autoencoder training
X_train, X_test = train_test_split(data, test_size=0.2, random_state=0)

In [93]:
# Set parameters for encoding dimension and L1 regularization strength
encoding_dim = 50
l1_value = 1e-6

In [94]:
# Build the autoencoder architecture
input_dim = data.shape[1]
input_layer = keras.Input(shape=(input_dim,))

# Encoder with L1 regularization
encoded = layers.Dense(512, activation="relu", activity_regularizer=regularizers.l1(l1_value))(input_layer)
encoded = layers.Dense(256, activation="relu")(encoded)
encoded = layers.Dense(encoding_dim, activation="sigmoid")(encoded)

# Decoder
decoded = layers.Dense(256, activation="relu")(encoded)
decoded = layers.Dense(512, activation="relu")(decoded)
decoded = layers.Dense(input_dim, activation="sigmoid")(decoded)

# Compile the autoencoder model
autoencoder = keras.Model(input_layer, decoded)
autoencoder.compile(optimizer="adam", loss="mse")

# Early stopping to prevent overfitting
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train autoencoder
history = autoencoder.fit(
    X_train, X_train,
    epochs=100, batch_size=32, verbose=1,
    validation_data=(X_test, X_test),
    callbacks=[early_stopping]
)

Epoch 1/100
40/40 [==============================] - 11s 250ms/step - loss: 0.1353 - val_loss: 0.1076
Epoch 2/100
40/40 [==============================] - 9s 232ms/step - loss: 0.1041 - val_loss: 0.0958
Epoch 3/100
40/40 [==============================] - 9s 227ms/step - loss: 0.0877 - val_loss: 0.0829
Epoch 4/100
40/40 [==============================] - 10s 240ms/step - loss: 0.0758 - val_loss: 0.0703
Epoch 5/100
40/40 [==============================] - 10s 240ms/step - loss: 0.0615 - val_loss: 0.0628
Epoch 6/100
40/40 [==============================] - 10s 244ms/step - loss: 0.0568 - val_loss: 0.0593
Epoch 7/100
40/40 [==============================] - 10s 239ms/step - loss: 0.0540 - val_loss: 0.0569
Epoch 8/100
40/40 [==============================] - 10s 242ms/step - loss: 0.0519 - val_loss: 0.0547
Epoch 9/100
40/40 [==============================] - 10s 240ms/step - loss: 0.0504 - val_loss: 0.0533
Epoch 10/100
40/40 [==============================] - 9s 236ms/step - loss: 0.0488 -

In [95]:
# Create encoder model to extract features and get feature importance from encoder
encoder = keras.Model(input_layer, encoded)
weights = encoder.layers[1].get_weights()[0]

In [96]:
weights

array([[ 0.00685879, -0.01163263,  0.0145897 , ..., -0.00042157,
         0.01179444,  0.00026602],
       [ 0.01112588,  0.00074701, -0.0170758 , ...,  0.00162638,
        -0.00029845, -0.00014893],
       [-0.01544441, -0.01437397, -0.01578079, ...,  0.00695208,
        -0.00961047, -0.00909195],
       ...,
       [-0.00612873,  0.01633504, -0.00952226, ...,  0.00454511,
        -0.00393211,  0.00030883],
       [ 0.00769359,  0.00378844,  0.0082393 , ...,  0.00978081,
         0.00831898, -0.00430642],
       [-0.01702774,  0.00895804, -0.01514908, ..., -0.01338666,
         0.01558216, -0.00401597]], dtype=float32)

In [98]:
# Compute average absolute weight for each input feature
feature_importance = np.mean(np.abs(weights), axis=1)
feature_importance

array([0.00935099, 0.00918005, 0.00894204, ..., 0.00895206, 0.0092099 ,
       0.00919418], dtype=float32)

In [99]:
# Assume feature_importance is already calculated (from encoder weights)
feature_importance = np.array(feature_importance)  # Convert to NumPy array

# Normalize feature importance values (scale between 0 and 1)
feature_importance = (feature_importance - feature_importance.min()) / (feature_importance.max() - feature_importance.min())

In [101]:
# Select features with normalized importance > certain threshold
selected_indices = np.where(feature_importance > 0.8)[0]
selected_indices

array([  105,   106,   649,   990,  1132,  1387,  1534,  2039,  2405,
        3486,  3882,  3951,  4238,  4659,  5080,  5302,  5993,  6057,
        6088,  6454,  6659,  7333,  7349,  7567,  7764,  8048,  8379,
        8448,  8544,  8817,  8912,  9209,  9321,  9337,  9563,  9635,
        9644,  9903, 10102, 10562, 10719, 10755, 11039, 11314, 11832,
       11937, 11951, 12071, 12111, 12209, 12390, 12565, 12684, 12707,
       12716, 13136, 13492, 13804, 13939, 14042, 14114, 15130, 15366,
       15848, 15958, 16883, 17611, 17660, 17897, 17961, 19052, 19177,
       19225, 19378, 19387, 19550])

In [102]:
len(selected_indices)

76

In [105]:
# Extract identifier (EpiRR) from column names using regex
data['identifier'] = data.index.map(dict)
data['identifier'] = data['identifier'].str.extract(r"(IHECRE\d+\.\d+)")
data

,chr1_840917_844971_n_1,chr1_999222_1001990_n_2,chr1_1024892_1028188_n_3,chr1_1135735_1138195_n_4,chr1_1217473_1220087_n_5,chr1_1297695_1303428_n_6,chr1_1356115_1362473_n_7,chr1_1398275_1400662_n_8,chr1_1418435_1435676_n_9,chr1_1539204_1541760_n_10,...,chrY_12661354_12663911_n_19764,chrY_13477999_13480655_n_19765,chrY_14523175_14526479_n_19766,chrY_19075455_19078099_n_19767,chrY_21072393_21083950_n_19768,chrY_21135972_21142739_n_19769,chrY_21173280_21177987_n_19770,chrY_21238808_21243547_n_19771,chrY_21247559_21259006_n_19772,identifier
0,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,IHECRE00000001.4
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,IHECRE00000002.3
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,IHECRE00000004.3
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,IHECRE00000008.3
4,0,0,0,0,0,0,0,1,0,0,...,1,1,0,0,0,0,0,1,0,IHECRE00000009.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1560,0,1,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,IHECRE00004888.1
1561,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,IHECRE00004894.1
1562,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,IHECRE00004897.1
1563,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,IHECRE00004900.1


In [110]:
# Select only important features and identifier column

indices = list(selected_indices)
indices.append(-1)
reduced_data = data.iloc[:,indices]
reduced_data

,chr1_8626494_8632469_n_106,chr1_8660187_8662868_n_107,chr1_44494801_44551424_n_650,chr1_109910177_109912864_n_991,chr1_151144480_151148078_n_1133,chr1_174937960_174944180_n_1388,chr1_200351331_200356959_n_1535,chr10_5941708_5946755_n_2040,chr10_69358345_69363626_n_2406,chr11_64648595_64656492_n_3487,...,chr7_149862694_149874974_n_17661,chr8_22972048_22976097_n_17898,chr8_28346320_28350409_n_17962,chr9_109316484_109320299_n_19053,chr9_121484568_121487442_n_19178,chr9_126397019_126401307_n_19226,chr9_133459278_133461688_n_19379,chr9_133786385_133790402_n_19388,chrX_30611897_30614616_n_19551,identifier
0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,IHECRE00000001.4
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,IHECRE00000002.3
2,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,IHECRE00000004.3
3,0,0,0,1,0,0,1,0,0,0,...,0,1,0,0,1,0,0,0,0,IHECRE00000008.3
4,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,IHECRE00000009.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1560,0,0,0,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,IHECRE00004888.1
1561,0,0,1,1,0,0,0,0,0,0,...,1,0,1,0,0,0,1,0,0,IHECRE00004894.1
1562,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,IHECRE00004897.1
1563,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,IHECRE00004900.1


In [111]:
ihec_data

,EpiRR,project,harmonized_biomaterial_type,harmonized_sample_label,harmonized_sample_ontology_intermediate,harmonized_sample_ontology_intermediate_color,harmonized_sample_disease_high,harmonized_sample_disease_intermediate,harmonized_EpiRR_status,epiATLAS_status,...,automated_experiments_ChIP-Seq_H3K36me3,automated_experiments_ChIP-Seq_H3K4me1,automated_experiments_ChIP-Seq_H3K4me3,automated_experiments_ChIP-Seq_H3K9me3,automated_experiments_WGBS_standard,automated_experiments_WGBS_PBAT,automated_experiments_RNA-Seq_mRNA-Seq,automated_experiments_RNA-Seq_total-RNA-Seq,epirr_id_without_version,EpiRR_ordering
0,IHECRE00001032.6,NIH Roadmap Epigenomics,primary tissue,Brain,brain,"185,126,30",Healthy/None,None,Partial,Partial,...,aa8df9df-e3b8-483c-9472-28f4928aa4b6,df1f1ce4-1a3a-4759-9431-f00274a7749b,e17c375b-300c-4045-9799-fc9bb5e8d78a,035e98a3-b88a-477c-9263-0b08641a3364,imputed,NaN,NaN,NaN,IHECRE00001032,1
1,IHECRE00001053.7,NIH Roadmap Epigenomics,primary tissue,Brain,brain,"185,126,30",Healthy/None,None,Partial,Partial,...,imputed,imputed,51ea052b-270e-4ae2-ab39-50b171bd58a5,imputed,imputed,NaN,NaN,NaN,IHECRE00001053,2
2,IHECRE00002063.1,CEEHRC,primary tissue,Brain Amygdala,brain,"185,126,30",Healthy/None,None,Complete,Complete,...,5cbd3cd5-dc33-49dc-ae71-41bb3114f33a,03cd26f6-8245-4dba-a43d-b085dc5fbdba,7418f6bc-6e40-4cd9-9c59-6013c4a60b00,29eb7dda-7f49-49e4-a68a-685ebc852d97,ad373be4-9739-4cbd-9f33-0ddd3ce45304,NaN,NaN,16319258-30fb-4402-a2c7-ae2b4f6f74be,IHECRE00002063,3
3,IHECRE00002086.1,CEEHRC,primary tissue,Brain Amygdala,brain,"185,126,30",Healthy/None,None,Complete,Complete,...,102990be-2f15-4965-9094-d6b665cc5fb0,469e2b5a-9cc0-4ba2-a1af-32ae688b6e56,e56e7efe-813c-4380-b8af-078cacdb962a,982b1a46-0bf8-457b-a213-1cec9bf76346,7ad1ee11-bdd1-4607-a385-226e88c6f940,NaN,NaN,da4afbf6-bcc1-4d3c-8eac-5e36d1a4b805,IHECRE00002086,4
4,IHECRE00001973.1,CEEHRC,primary tissue,Brain Amygdala,brain,"185,126,30",Healthy/None,None,Complete,Complete,...,f006c884-d6f7-470d-8586-314761e38a9f,fa7392b1-17c1-4691-ba8f-d211a0f93db5,12487572-adf6-40c8-9745-66a141c14768,6e93a1e0-8828-4ce5-83f8-36249902bda2,0fad140f-c9a5-4da6-b789-485e11b7bdf3,NaN,NaN,cd02f672-1b6d-4841-83d9-c7d187a2a7f5,IHECRE00001973,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2274,IHECRE00000975.8,NIH Roadmap Epigenomics,primary cell,Melanocyte Foreskin,melanocyte,"113,41,32",Healthy/None,None,Partial,Partial,...,401fde27-8902-40c0-8a71-c1861d67a913,28dce804-616e-461a-a241-f86f38849801,97a1e865-f6f5-4bb4-a302-8bb6deedc0ef,0cf75112-954a-4d05-98c6-8191f76b9e0b,imputed,NaN,NaN,NaN,IHECRE00000975,2275
2275,IHECRE00001026.7,NIH Roadmap Epigenomics,primary cell,Melanocyte Foreskin,melanocyte,"113,41,32",Healthy/None,None,Partial,Complete_imputed,...,81227768-8af0-4c8e-8b0b-e1308eaefa55,12e4d18a-69a4-49cf-aa63-7cd9c6b4f158,160994fa-0800-4eaf-ac95-c9c39242a36b,997e79ff-8ff2-489c-8380-15cef3b2293f,imputed,NaN,fafd4afa-b115-4895-922c-141e82bffa60,NaN,IHECRE00001026,2276
2276,IHECRE00000928.7,NIH Roadmap Epigenomics,primary tissue,Ovarian Tissue,ovary,"181,104,96",Healthy/None,None,Complete,Partial,...,NaN,NaN,NaN,NaN,NaN,49594f8a-8dd8-47f3-989d-3aa48f1dd6c4,d1c68642-5108-4497-b725-e555cc161c3b,NaN,IHECRE00000928,2277
2277,IHECRE00004688.5,ENCODE,primary tissue,Uterine Tissue,uterus,"135,105,48",Healthy/None,None,Partial,Complete_imputed,...,imputed,0785d70e-043f-4e7a-bc91-6d59853273fa,a1984693-dd60-4afe-9120-d47d69e32402,ce89c637-c90f-4659-b060-617a76f443f9,imputed,NaN,NaN,e0902bfa-5f1d-4320-a381-be7227f1b333,IHECRE00004688,2278


In [113]:
# Merge reduced features with metadata for label information
labeled_data = pd.merge(reduced_data, ihec_data[['EpiRR', 'harmonized_sample_ontology_intermediate']], left_on='identifier', right_on='EpiRR', how='inner')
labeled_data

,chr1_8626494_8632469_n_106,chr1_8660187_8662868_n_107,chr1_44494801_44551424_n_650,chr1_109910177_109912864_n_991,chr1_151144480_151148078_n_1133,chr1_174937960_174944180_n_1388,chr1_200351331_200356959_n_1535,chr10_5941708_5946755_n_2040,chr10_69358345_69363626_n_2406,chr11_64648595_64656492_n_3487,...,chr8_28346320_28350409_n_17962,chr9_109316484_109320299_n_19053,chr9_121484568_121487442_n_19178,chr9_126397019_126401307_n_19226,chr9_133459278_133461688_n_19379,chr9_133786385_133790402_n_19388,chrX_30611897_30614616_n_19551,identifier,EpiRR,harmonized_sample_ontology_intermediate
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,IHECRE00000001.4,IHECRE00000001.4,mammary gland cell line
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,IHECRE00000002.3,IHECRE00000002.3,myeloid cell
2,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,1,0,0,IHECRE00000004.3,IHECRE00000004.3,neutrophil
3,0,0,0,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,IHECRE00000008.3,IHECRE00000008.3,macrophage
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,IHECRE00000009.3,IHECRE00000009.3,T cell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,IHECRE00004888.1,IHECRE00004888.1,connective tissue cell
1466,0,0,1,1,0,0,0,0,0,0,...,1,0,0,0,1,0,0,IHECRE00004894.1,IHECRE00004894.1,neural progenitor cell
1467,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,IHECRE00004897.1,IHECRE00004897.1,placenta
1468,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,IHECRE00004900.1,IHECRE00004900.1,placenta


In [115]:
# Identify rare classes (classes with only 1 instance)
rare_classes = labeled_data['harmonized_sample_ontology_intermediate'].value_counts()[labeled_data['harmonized_sample_ontology_intermediate'].value_counts() == 1].index
rare_classes

Index(['lung', 'neural cell', 'keratinocyte', 'liver',
       'fibroblast derived cell line', 'mononuclear cell',
       'neural progenitor cell', 'trophoblast'],
      dtype='object')

In [117]:
# Manually place rare classes in the test set
test_samples = labeled_data[labeled_data['harmonized_sample_ontology_intermediate'].isin(rare_classes)]
test_samples

,chr1_8626494_8632469_n_106,chr1_8660187_8662868_n_107,chr1_44494801_44551424_n_650,chr1_109910177_109912864_n_991,chr1_151144480_151148078_n_1133,chr1_174937960_174944180_n_1388,chr1_200351331_200356959_n_1535,chr10_5941708_5946755_n_2040,chr10_69358345_69363626_n_2406,chr11_64648595_64656492_n_3487,...,chr8_28346320_28350409_n_17962,chr9_109316484_109320299_n_19053,chr9_121484568_121487442_n_19178,chr9_126397019_126401307_n_19226,chr9_133459278_133461688_n_19379,chr9_133786385_133790402_n_19388,chrX_30611897_30614616_n_19551,identifier,EpiRR,harmonized_sample_ontology_intermediate
133,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,IHECRE00000277.2,IHECRE00000277.2,mononuclear cell
242,0,0,0,0,0,0,0,1,1,0,...,1,0,0,0,1,0,0,IHECRE00000964.7,IHECRE00000964.7,lung
260,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,IHECRE00000992.7,IHECRE00000992.7,neural cell
262,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,IHECRE00000994.7,IHECRE00000994.7,keratinocyte
293,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,IHECRE00001031.6,IHECRE00001031.6,liver
298,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,IHECRE00001041.8,IHECRE00001041.8,fibroblast derived cell line
1466,0,0,1,1,0,0,0,0,0,0,...,1,0,0,0,1,0,0,IHECRE00004894.1,IHECRE00004894.1,neural progenitor cell
1469,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,IHECRE00004903.1,IHECRE00004903.1,trophoblast


In [119]:
# Remove rare classes from training data
labeled_data = labeled_data.drop(test_samples.index)
labeled_data

,chr1_8626494_8632469_n_106,chr1_8660187_8662868_n_107,chr1_44494801_44551424_n_650,chr1_109910177_109912864_n_991,chr1_151144480_151148078_n_1133,chr1_174937960_174944180_n_1388,chr1_200351331_200356959_n_1535,chr10_5941708_5946755_n_2040,chr10_69358345_69363626_n_2406,chr11_64648595_64656492_n_3487,...,chr8_28346320_28350409_n_17962,chr9_109316484_109320299_n_19053,chr9_121484568_121487442_n_19178,chr9_126397019_126401307_n_19226,chr9_133459278_133461688_n_19379,chr9_133786385_133790402_n_19388,chrX_30611897_30614616_n_19551,identifier,EpiRR,harmonized_sample_ontology_intermediate
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,IHECRE00000001.4,IHECRE00000001.4,mammary gland cell line
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,IHECRE00000002.3,IHECRE00000002.3,myeloid cell
2,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,1,0,0,IHECRE00000004.3,IHECRE00000004.3,neutrophil
3,0,0,0,1,0,0,1,0,0,0,...,0,0,1,0,0,0,0,IHECRE00000008.3,IHECRE00000008.3,macrophage
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,IHECRE00000009.3,IHECRE00000009.3,T cell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1463,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,IHECRE00004885.1,IHECRE00004885.1,connective tissue cell
1464,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,IHECRE00004886.1,IHECRE00004886.1,mesoderm-derived structure
1465,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,IHECRE00004888.1,IHECRE00004888.1,connective tissue cell
1467,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,IHECRE00004897.1,IHECRE00004897.1,placenta


In [120]:
# Split dataset into features and labels
X = labeled_data.iloc[:, :-3] 
y = labeled_data["harmonized_sample_ontology_intermediate"]  # Target labels

# Extract test set for rare classes
X_test_extra = test_samples.iloc[:, :-3]
y_test_extra = test_samples["harmonized_sample_ontology_intermediate"] 

In [121]:
# Split dataset into training and testing sets (80% train, 20% test)
# Stratified split for balanced class distribution
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [122]:
# Add rare classes to test set
X_test = pd.concat([X_test, X_test_extra])
y_test = pd.concat([y_test, y_test_extra])

In [148]:
# Define parameter grid for MLP classifier
param_grid = {
    'hidden_layer_sizes': [(10,), (50,), (100,), (50, 50), (64, 32), (128, 64, 32)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001, 0.01],
    'batch_size': [20, 32, 64],
    'learning_rate': ['constant', 'adaptive'],
    'learning_rate_init': [0.001, 0.01],
}

# Initialize MLP model
mlp = MLPClassifier(max_iter=1000, random_state=111)

# Perform grid search with 5 fold cross-validation
grid_search = GridSearchCV(
    estimator=mlp,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',   
    n_jobs=-1,            
    verbose=2
)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 864 candidates, totalling 4320 fits


/Users/gayathrimacharla/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/Users/gayathrimacharla/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/gayathrimacharla/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/gayathrimacharla/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/gayathrimacharla/ana

[CV] END activation=relu, alpha=0.0001, batch_size=20, hidden_layer_sizes=(10,), learning_rate=constant, learning_rate_init=0.01, solver=adam; total time=   9.3s
[CV] END activation=relu, alpha=0.0001, batch_size=20, hidden_layer_sizes=(10,), learning_rate=constant, learning_rate_init=0.01, solver=adam; total time=  11.8s
[CV] END activation=relu, alpha=0.0001, batch_size=20, hidden_layer_sizes=(10,), learning_rate=constant, learning_rate_init=0.01, solver=adam; total time=   8.9s
[CV] END activation=relu, alpha=0.0001, batch_size=20, hidden_layer_sizes=(10,), learning_rate=constant, learning_rate_init=0.01, solver=sgd; total time=  37.2s
[CV] END activation=relu, alpha=0.0001, batch_size=20, hidden_layer_sizes=(10,), learning_rate=adaptive, learning_rate_init=0.001, solver=adam; total time=  53.2s
[CV] END activation=relu, alpha=0.0001, batch_size=20, hidden_layer_sizes=(50,), learning_rate=constant, learning_rate_init=0.001, solver=adam; total time=  32.8s
[CV] END activation=relu, a

/Users/gayathrimacharla/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/gayathrimacharla/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/gayathrimacharla/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/gayathrimacharla/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  war

[CV] END activation=relu, alpha=0.0001, batch_size=20, hidden_layer_sizes=(10,), learning_rate=constant, learning_rate_init=0.001, solver=sgd; total time= 1.3min
[CV] END activation=relu, alpha=0.0001, batch_size=20, hidden_layer_sizes=(10,), learning_rate=adaptive, learning_rate_init=0.01, solver=adam; total time=  11.6s
[CV] END activation=relu, alpha=0.0001, batch_size=20, hidden_layer_sizes=(10,), learning_rate=adaptive, learning_rate_init=0.01, solver=adam; total time=  14.0s
[CV] END activation=relu, alpha=0.0001, batch_size=20, hidden_layer_sizes=(10,), learning_rate=adaptive, learning_rate_init=0.01, solver=sgd; total time= 1.0min
[CV] END activation=relu, alpha=0.0001, batch_size=20, hidden_layer_sizes=(50,), learning_rate=constant, learning_rate_init=0.001, solver=sgd; total time= 1.7min
[CV] END activation=relu, alpha=0.0001, batch_size=20, hidden_layer_sizes=(50,), learning_rate=adaptive, learning_rate_init=0.01, solver=sgd; total time=  46.5s
[CV] END activation=relu, alph

...

[CV] END activation=tanh, alpha=0.0001, batch_size=20, hidden_layer_sizes=(100,), learning_rate=adaptive, learning_rate_init=0.001, solver=sgd; total time= 1.4min
[CV] END activation=tanh, alpha=0.0001, batch_size=20, hidden_layer_sizes=(50, 50), learning_rate=constant, learning_rate_init=0.01, solver=sgd; total time=  23.5s
[CV] END activation=tanh, alpha=0.0001, batch_size=20, hidden_layer_sizes=(50, 50), learning_rate=adaptive, learning_rate_init=0.001, solver=sgd; total time= 1.2min
[CV] END activation=tanh, alpha=0.0001, batch_size=20, hidden_layer_sizes=(64, 32), learning_rate=constant, learning_rate_init=0.01, solver=sgd; total time=  18.3s
[CV] END activation=tanh, alpha=0.0001, batch_size=20, hidden_layer_sizes=(64, 32), learning_rate=adaptive, learning_rate_init=0.001, solver=sgd; total time= 1.2min
[CV] END activation=tanh, alpha=0.0001, batch_size=20, hidden_layer_sizes=(128, 64, 32), learning_rate=constant, learning_rate_init=0.01, solver=adam; total time=   8.4s
[CV] END 

/Users/gayathrimacharla/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/gayathrimacharla/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/gayathrimacharla/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/gayathrimacharla/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  war

[CV] END activation=tanh, alpha=0.0001, batch_size=32, hidden_layer_sizes=(50, 50), learning_rate=constant, learning_rate_init=0.01, solver=sgd; total time=  54.0s
[CV] END activation=tanh, alpha=0.0001, batch_size=32, hidden_layer_sizes=(50, 50), learning_rate=adaptive, learning_rate_init=0.001, solver=adam; total time=  45.1s
[CV] END activation=tanh, alpha=0.0001, batch_size=32, hidden_layer_sizes=(50, 50), learning_rate=adaptive, learning_rate_init=0.001, solver=sgd; total time= 3.1min
[CV] END activation=tanh, alpha=0.0001, batch_size=32, hidden_layer_sizes=(64, 32), learning_rate=adaptive, learning_rate_init=0.001, solver=adam; total time=  41.2s
[CV] END activation=tanh, alpha=0.0001, batch_size=32, hidden_layer_sizes=(64, 32), learning_rate=adaptive, learning_rate_init=0.001, solver=adam; total time=  45.9s
[CV] END activation=tanh, alpha=0.0001, batch_size=32, hidden_layer_sizes=(64, 32), learning_rate=adaptive, learning_rate_init=0.001, solver=sgd; total time= 3.2min
[CV] END

/Users/gayathrimacharla/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/gayathrimacharla/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/gayathrimacharla/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/gayathrimacharla/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  war

GridSearchCV(cv=5, estimator=MLPClassifier(max_iter=1000, random_state=111),
             n_jobs=-1,
             param_grid={'activation': ['relu', 'tanh'],
                         'alpha': [0.0001, 0.001, 0.01],
                         'batch_size': [20, 32, 64],
                         'hidden_layer_sizes': [(10,), (50,), (100,), (50, 50),
                                                (64, 32), (128, 64, 32)],
                         'learning_rate': ['constant', 'adaptive'],
                         'learning_rate_init': [0.001, 0.01],
                         'solver': ['adam', 'sgd']},
             scoring='accuracy', verbose=2)

In [149]:
# Print best parameters and accuracy score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validated Accuracy:", grid_search.best_score_)

Best Parameters: {'activation': 'relu', 'alpha': 0.001, 'batch_size': 32, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'learning_rate_init': 0.01, 'solver': 'sgd'}
Best Cross-Validated Accuracy: 0.8700011004732036


In [155]:
# Train MLP with selected parameters (manually set here based on grid search results)
tuned_model = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='sgd', alpha=0.001, batch_size=32,
                learning_rate='constant', learning_rate_init=0.01, max_iter=1000, random_state=111, n_iter_no_change=10)

# Train the model
tuned_model.fit(X_train, y_train)

MLPClassifier(alpha=0.001, batch_size=32, learning_rate_init=0.01,
              max_iter=1000, random_state=111, solver='sgd')

In [156]:
# Make predictions on the test set
y_pred_tuned = tuned_model.predict(X_test)

In [157]:
# Evaluate model performance
accuracy_tuned = accuracy_score(y_test, y_pred_tuned)
print(f"Classification Accuracy: {accuracy_tuned:.4f}")

Classification Accuracy: 0.8571


In [158]:
# Generate and print a detailed classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_tuned))

Classification Report:
                                      precision    recall  f1-score   support

                              T cell       0.95      0.93      0.94        41
                               brain       0.98      0.97      0.98        62
                    cancer cell line       0.00      0.00      0.00         1
                     capillary blood       0.00      0.00      0.00         2
                               colon       1.00      0.20      0.33         5
              connective tissue cell       0.67      1.00      0.80         4
                      dendritic cell       0.00      0.00      0.00         1
                    digestive system       0.40      1.00      0.57         2
                endo-epithelial cell       0.33      0.50      0.40         2
epithelial cell of endometrial gland       0.00      0.00      0.00         1
                 extraembryonic cell       0.57      1.00      0.73         4
        fibroblast derived cell line    

/Users/gayathrimacharla/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/gayathrimacharla/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/gayathrimacharla/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", l

[CV] END activation=tanh, alpha=0.001, batch_size=64, hidden_layer_sizes=(128, 64, 32), learning_rate=constant, learning_rate_init=0.01, solver=adam; total time=  14.1s
[CV] END activation=tanh, alpha=0.001, batch_size=64, hidden_layer_sizes=(128, 64, 32), learning_rate=constant, learning_rate_init=0.01, solver=sgd; total time= 1.6min
[CV] END activation=tanh, alpha=0.001, batch_size=64, hidden_layer_sizes=(128, 64, 32), learning_rate=adaptive, learning_rate_init=0.001, solver=sgd; total time= 3.2min
[CV] END activation=tanh, alpha=0.01, batch_size=20, hidden_layer_sizes=(10,), learning_rate=constant, learning_rate_init=0.01, solver=adam; total time=  11.1s
[CV] END activation=tanh, alpha=0.01, batch_size=20, hidden_layer_sizes=(10,), learning_rate=constant, learning_rate_init=0.01, solver=sgd; total time= 1.0min
[CV] END activation=tanh, alpha=0.01, batch_size=20, hidden_layer_sizes=(10,), learning_rate=adaptive, learning_rate_init=0.001, solver=adam; total time= 1.3min
[CV] END activ